In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
train_df = pd.read_csv('/kaggle/input/shutterstock-dataset-for-ai-vs-human-gen-image/train.csv')
train_df.head()

,Unnamed: 0,file_name,label
0,0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1


In [22]:
train_df = train_df.drop(columns=["Unnamed: 0"])
train_df.shape

(79950, 2)

In [23]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.utils import to_categorical


In [24]:
IMG_SIZE = 224
BATCH_SIZE = 32
VAL_SPLIT = 0.2
SEED = 42

In [25]:
BASE_PATH = "/kaggle/input/shutterstock-dataset-for-ai-vs-human-gen-image/"

file_paths = train_df["file_name"].apply(
    lambda x: BASE_PATH + x
).values


labels = train_df["label"].values


### Shuffling indices

In [26]:
np.random.seed(SEED)

indices = np.arange(len(file_paths))
np.random.shuffle(indices)

file_paths = file_paths[indices]
labels = labels[indices]


### Train & validation split

In [27]:
val_size = int(len(file_paths) * VAL_SPLIT)

val_paths = file_paths[:val_size]
val_labels = labels[:val_size]

train_paths = file_paths[val_size:]
train_labels = labels[val_size:]


## Image loading function

In [28]:
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0

    label = tf.cast(label, tf.float32)
    
    return image, label


### Creating Train dataset

In [29]:
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))

train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)


### Creating validation dataset

In [30]:
val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

val_ds = val_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)


In [31]:
for images, labels in train_ds.take(1):
    print(images.shape)
    print(labels.shape)


(32, 224, 224, 3)
(32,)


## Transfer learning

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, Model

base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False


In [ ]:
inputs = layers.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)

x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(1, activation='sigmoid')(x) 

model = Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor="val_loss",
    save_best_only=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stop, checkpoint]
)

model.save("/kaggle/working/final_model.keras")



### these were epochs, epoch2's model is saved and will furthur finetune..
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.7020 - loss: 0.5749

1999/1999 ━━━━━━━━━━━━━━━━━━━━ 234s 104ms/step - accuracy: 0.7020 - loss: 0.5749 - val_accuracy: 0.7859 - val_loss: 0.4665

Epoch 2/20
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 97s 49ms/step - accuracy: 0.7607 - loss: 0.5038 - val_accuracy: 0.7988 - val_loss: 0.4524

Epoch 3/20
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 94s 47ms/step - accuracy: 0.7714 - loss: 0.4820 - val_accuracy: 0.7448 - val_loss: 0.5165

Epoch 4/20
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 95s 47ms/step - accuracy: 0.7810 - loss: 0.4686 - val_accuracy: 0.6436 - val_loss: 0.6251

Epoch 5/20
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 95s 47ms/step - accuracy: 0.7879 - loss: 0.4588 - val_accuracy: 0.5876 - val_loss: 0.7110

## Finetuning best model

In [30]:
model = tf.keras.models.load_model("best_model.keras")
base_model = model.get_layer('efficientnetb0')


In [31]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,036,712 (30.66 MB)

 Trainable params: 1,826,913 (6.97 MB)

 Non-trainable params: 2,555,971 (9.75 MB)

 Optimizer params: 3,653,828 (13.94 MB)

In [32]:
for layer in base_model.layers:
    layer.trainable = False


In [33]:
trainable_count = sum([1 for layer in base_model.layers if layer.trainable])
print("Trainable layers in backbone:", trainable_count)


Trainable layers in backbone: 0


In [34]:
base_model.summary()


Model: "efficientnetb0"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_4         │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_2     │ (None, 224, 224,  │          7 │ rescaling_4[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_5         │ (None, 224, 224,  │          0 │ normalization_2[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_5[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

### Unfreezing last 10 layers...

In [35]:
for layer in base_model.layers[-10:]:
    layer.trainable = True


In [36]:
trainable_count = sum([1 for layer in base_model.layers if layer.trainable])
print("Trainable layers in backbone:", trainable_count)


Trainable layers in backbone: 10


In [37]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [38]:
len(base_model.layers)


238

In [39]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor="val_loss",
    save_best_only=True
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop, checkpoint]
)
model.save("/kaggle/working/finetuned_model.keras")

Epoch 1/10


I0000 00:00:1770893363.268300    1134 service.cc:152] XLA service 0x7994d40140a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770893363.268351    1134 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1770893363.268356    1134 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1770893365.639406    1134 cuda_dnn.cc:529] Loaded cuDNN version 91002
2026-02-12 10:49:31.455904: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:49:31.599045: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:49:31.938694: E external/local_xl

1997/1999 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7551 - loss: 0.5195

2026-02-12 10:51:14.129096: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:14.270483: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:14.601993: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:14.742755: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:15.469549: E external/local_xla/xla/stream_

1999/1999 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7551 - loss: 0.5195

2026-02-12 10:51:55.584008: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:55.725406: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:56.054668: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:56.195195: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 10:51:56.894390: E external/local_xla/xla/stream_

1999/1999 ━━━━━━━━━━━━━━━━━━━━ 168s 70ms/step - accuracy: 0.7551 - loss: 0.5195 - val_accuracy: 0.8005 - val_loss: 0.4454
Epoch 2/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 95s 47ms/step - accuracy: 0.7661 - loss: 0.4950 - val_accuracy: 0.7313 - val_loss: 0.5212
Epoch 3/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 94s 47ms/step - accuracy: 0.7779 - loss: 0.4758 - val_accuracy: 0.5580 - val_loss: 0.7212
Epoch 4/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 94s 47ms/step - accuracy: 0.7817 - loss: 0.4655 - val_accuracy: 0.5245 - val_loss: 0.8063


## Comapring non tuned and tuned model...

## best epoch was saved so
non tuned:

Epoch 2/20 1999/1999 ━━━━━━━━━━━━━━━━━━━━ 97s 49ms/step - accuracy: 0.7607 - loss: 0.5038 - val_accuracy: 0.7988 - val_loss: 0.4524

tuned:

1999/1999 ━━━━━━━━━━━━━━━━━━━━ 168s 70ms/step - accuracy: 0.7551 - loss: 0.5195 - val_accuracy: 0.8005 - val_loss: 0.4454

## Not much difference but fine tuned to last 10 layers is better...
saved with name - 'finetuned_model.keras'

# CNN from scratch

In [32]:
for images, labels in train_ds.take(1):
    print(images.shape)
    print(labels.shape)


(32, 224, 224, 3)
(32,)


In [33]:
data_aug =Sequential([
    layers.RandomRotation(0.03),
    layers.RandomZoom(0.08),
    layers.RandomTranslation(0.05, 0.05),
])

In [34]:
model = Sequential([

    layers.Input(shape=(224, 224, 3)),

    data_aug,

    layers.Conv2D(32,(3,3),activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),

    layers.GlobalAveragePooling2D(),

    layers.Dense(units=128,activation='relu'),

    layers.Dense(units=1,activation='sigmoid')
    
])

In [35]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,225 (110.25 KB)

 Trainable params: 28,033 (109.50 KB)

 Non-trainable params: 192 (768.00 B)

In [36]:
model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])

In [37]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor="val_loss",
    save_best_only=True
)

history_scratchcnn = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop, checkpoint]
)
model.save("/kaggle/working/cnn_model.keras")

Epoch 1/10


E0000 00:00:1770895947.494717    2018 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_3_1/dropout_2_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1770895948.290930    2087 cuda_dnn.cc:529] Loaded cuDNN version 91002


1999/1999 ━━━━━━━━━━━━━━━━━━━━ 261s 128ms/step - accuracy: 0.8759 - loss: 0.2944 - val_accuracy: 0.8627 - val_loss: 0.3428
Epoch 2/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 251s 126ms/step - accuracy: 0.9191 - loss: 0.2044 - val_accuracy: 0.8838 - val_loss: 0.2892
Epoch 3/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 251s 125ms/step - accuracy: 0.9333 - loss: 0.1718 - val_accuracy: 0.8620 - val_loss: 0.3118
Epoch 4/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 251s 125ms/step - accuracy: 0.9397 - loss: 0.1543 - val_accuracy: 0.9071 - val_loss: 0.2325
Epoch 5/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 251s 126ms/step - accuracy: 0.9473 - loss: 0.1402 - val_accuracy: 0.9159 - val_loss: 0.2258
Epoch 6/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 250s 125ms/step - accuracy: 0.9505 - loss: 0.1296 - val_accuracy: 0.9114 - val_loss: 0.2284
Epoch 7/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 250s 125ms/step - accuracy: 0.9545 - loss: 0.1220 - val_accuracy: 0.8992 - val_loss: 0.2643
Epoch 8/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 250s 125ms/step - accuracy: 0.9

In [38]:
cnnmodel = tf.keras.models.load_model("cnn_model.keras")
cnnmodel.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,293 (329.27 KB)

 Trainable params: 28,033 (109.50 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 56,068 (219.02 KB)

# Evaulation on scratch cnn 

In [39]:
test_ds = pd.read_csv('/kaggle/input/shutterstock-dataset-for-ai-vs-human-gen-image/test_v2_labels.csv')
test_ds.head()

,id,label
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg,1.0
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg,1.0
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg,0.0
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg,0.0
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg,0.0


In [41]:
import numpy as np
import tensorflow as tf

BASE_PATH = "/kaggle/input/shutterstock-dataset-for-ai-vs-human-gen-image/"


test_paths = test_ds["id"].apply(lambda x: BASE_PATH + x).values
test_labels = test_ds["label"].values.astype(np.float32)


In [42]:
def load_image_test(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, tf.cast(label, tf.float32)

# Create dataset
test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_image_test, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


In [45]:
preds = cnnmodel.predict(test_ds)
preds_class = (preds > 0.5).astype(int)  # Binary thresholding


from sklearn.metrics import accuracy_score, confusion_matrix

y_true = test_labels
y_pred = preds_class.flatten()

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


174/174 ━━━━━━━━━━━━━━━━━━━━ 31s 175ms/step
Accuracy: 0.5799638989169675
Confusion Matrix:
 [[2464  194]
 [2133  749]]


In [46]:
ftmodel = tf.keras.models.load_model("finetuned_model.keras")
ftmodel.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,830,856 (26.06 MB)

 Trainable params: 1,223,985 (4.67 MB)

 Non-trainable params: 3,158,899 (12.05 MB)

 Optimizer params: 2,447,972 (9.34 MB)

In [47]:
preds = ftmodel.predict(test_ds)
preds_class = (preds > 0.5).astype(int)  # Binary thresholding


from sklearn.metrics import accuracy_score, confusion_matrix

y_true = test_labels
y_pred = preds_class.flatten()

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


I0000 00:00:1770899360.013121    2087 service.cc:152] XLA service 0x7b7bdcf83130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770899360.013171    2087 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1770899360.013177    2087 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2026-02-12 12:29:25.301130: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 12:29:25.444916: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 12:29:25.785516: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time

173/174 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step

2026-02-12 12:30:07.170227: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 12:30:07.305023: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 12:30:07.609490: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 12:30:07.749957: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-12 12:30:08.433422: E external/local_xla/xla/stream_

174/174 ━━━━━━━━━━━━━━━━━━━━ 54s 239ms/step
Accuracy: 0.5703971119133574
Confusion Matrix:
 [[1928  730]
 [1650 1232]]
